In [36]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import HdpModel, CoherenceModel
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Muhammad Ade
[nltk_data]     Aulia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Muhammad Ade
[nltk_data]     Aulia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# Load data from CSV file
df = pd.read_csv('../data/dataHasilPreprocessing/hasilPreprocessing1.csv')  # Ganti dengan path file Anda
df = df.drop_duplicates(subset=['Ulasan'])
df = df.dropna()
df

,Ulasan,Sentimen
0,ulas,Negatif
2,bahan aja warna navy nya beda,Negatif
3,kasih catat order warna kuning navy baca kirim...,Negatif
4,kecil,Negatif
5,tau layan,Negatif
...,...,...
989,cocok,Negatif
990,barang selamat terima kasih bahan celana nya s...,Negatif
992,pesan cuna barang rusak coba konfirmasi tanggap,Negatif
993,karet pinggang nya kencang,Negatif


In [38]:
# 2. Text Vectorization
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Ulasan'])

In [39]:
# Label Encoding
le = LabelEncoder()
y = le.fit_transform(df['Sentimen'])

In [40]:
# 3. Ekstraksi Fitur (Hierarchical Dirichlet Process)
texts = [ulasan.split() for ulasan in df['Ulasan']]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
hdp_model = HdpModel(corpus, id2word=dictionary)

In [41]:
# Menampilkan topik HDP
topik_hdp = hdp_model.show_topics(formatted=False)
print("Topik HDP:")
for topik in topik_hdp:
    print(topik)

Topik HDP:
(0, [('maksa', 0.0045824006172974436), ('hanafashion', 0.004156158767931116), ('kertas', 0.004090166973172764), ('owner', 0.004079411976214477), ('in', 0.00401501167189705), ('fotoo', 0.003964421648274325), ('longgar', 0.003913759191617363), ('terima', 0.0038906254307967303), ('ngga', 0.003781565828896018), ('cepat', 0.003765555574432667), ('putiny', 0.003719900305769384), ('batalin', 0.0036669888721990165), ('resi', 0.003594678677659554), ('warna', 0.0035550579028884094), ('sdikit', 0.003415632392317027), ('cmn', 0.0034139895757075373), ('jaitanna', 0.003406109470270391), ('yaaa', 0.0033538966635554748), ('body', 0.0033030681048099083), ('samping', 0.003291776643538051)])
(1, [('bnyk', 0.005984647176811784), ('ma', 0.004530514935379143), ('gw', 0.004180732309356786), ('pa', 0.00400026586580884), ('buwanget', 0.003991844023988196), ('recomanded', 0.003956065858211552), ('jaitanna', 0.0039285986660769915), ('sp', 0.0039046604604529695), ('outernya', 0.0036806944407124214), ('

In [46]:
# Menghitung skor koherensi
coherence_model_hdp = CoherenceModel(model=hdp_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_scores = coherence_model_hdp.get_coherence_per_topic()

# Membuat DataFrame untuk skor koherensi
num_topics = list(range(1, len(coherence_scores) + 1))
coherence_df = pd.DataFrame({'Jumlah Topik': num_topics, 'Skor Coherence': coherence_scores})

print("\nSkor Koherensi untuk tiap topik:")
coherence_df.head(20)


Skor Koherensi untuk tiap topik:


,Jumlah Topik,Skor Coherence
0,1,0.761492
1,2,0.747949
2,3,0.771252
3,4,0.755569
4,5,0.760930
5,6,0.760552
6,7,0.802551
7,8,0.804538
8,9,0.806186
9,10,0.786098


In [43]:
# Mengonversi HDP menjadi fitur
X_hdp = np.zeros((len(corpus), hdp_model.m_T))
for i, doc in enumerate(corpus):
    for topic_id, weight in hdp_model[doc]:
        if topic_id < hdp_model.m_T:
            X_hdp[i, topic_id] = weight

In [44]:
# 4. Klasifikasi menggunakan SVM
X_train, X_test, y_train, y_test = train_test_split(X_hdp, y, test_size=0.2, random_state=42)
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

In [45]:
# 5. Evaluasi classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Classification Report:
              precision    recall  f1-score   support

     Negatif       0.00      0.00      0.00        23
     Positif       0.88      1.00      0.94       167

    accuracy                           0.88       190
   macro avg       0.44      0.50      0.47       190
weighted avg       0.77      0.88      0.82       190



c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier